# Network Traffic Analysis Pipeline

## 0. Setup Environment

In [1]:
%pip install pyshark
%pip install nest_asyncio
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Allow the notebook to work assynchronously.

In [2]:
import nest_asyncio
nest_asyncio.apply()

## 1. Data Gathering

In [3]:
dataset_files = ['data/2024-10-23-Redline-Stealer-infection-traffic.pcap']

## 2. Data Preprocessing

1. Import the desired preprocessor.

In [4]:
from src.preprocessors.pcap_preprocessor import PcapPreprocessor

preprocessor = PcapPreprocessor()

2. Load the datasets into a dataframe.

In [5]:
base_data = preprocessor.load_datasets(dataset_files)
print(base_data)

Processed 1000 packets


KeyboardInterrupt: 

3. Preprocess the dataframe.

In [ ]:
processed_data = preprocessor.preprocess_dataframe(base_data)

4. Partition the dataframe into training and testing sets.

In [ ]:
base_training_percentage = 0.6
online_training_percentage = 0.2
validation_percentage = 0.2

#training_data, online_training_data, validation_data = preprocessor.split_dataframe(processed_data, base_training_percentage, online_training_percentage, validation_percentage)

## 3. Model Training

1. Select the desired models.

In [ ]:
from src.models.offline import OfflineModel
from src.models.online import OnlineModel

model_list = [OfflineModel(), OnlineModel()]

2. Train the models according to their type.

In [ ]:
for model in model_list:
    model.train(training_data)
    if model.is_online():
        model.predict_batch(online_training_data)

## 4. Model Evaluation

1. Get the predictions from the models into a dataframe.

In [ ]:
# Header
header = ['expected']
for model in model_list:
    header.append(model.get_name())
validation_results = [header]

# Results
for index, row in validation_data.iterrows():
    line = [row['label']]
    for model in model_list:
        line.append(model.predict(row))
    validation_results.append(line)
 
df = pd.DataFrame(validation_results)

2. Select the desired evaluator.

In [ ]:
from src.evaluators.standard_evaluator import Evaluator

evaluator = Evaluator()

3. Evaluate the models using the predictions.

In [ ]:
results = evaluator.evaluate(df)

4. Analyze the results.

In [ ]:
print(results)